In [71]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [81]:
# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Define batch size and number of batches
batch_size = len(data) // 2
num_batches = len(data[0].split(',')) // batch_size

# Define player data
players = []
for i in range(0, len(data), 3):
    player = {
        'defense_ranks': np.array([int(j) for j in data[i].split(',')], dtype=np.float32),
        'fantasy_scores': np.array([float(j) for j in data[i+1].split(',')], dtype=np.float32),
        'skill_score': np.array([float(j) for j in data[i+2].split(',')], dtype=np.float32),
    }
    players.append(player)

# Process inputs and targets for each player
X_train, X_test, Y_train, Y_test = [], [], [], []
 
for player in players:
    player_inputs = torch.from_numpy(player['defense_ranks'][:10]).view(1, 10, 1).float()
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['skill_score'][:10]).view(1, 10, 1).float()), dim=2)

    player_test_inputs = torch.from_numpy(player['defense_ranks'][-5:]).view(1, 5, 1).float()
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['skill_score'][-5:]).view(1, 5, 1).float()), dim=2)

    X_train.append(player_inputs)
    X_test.append(player_test_inputs)
    Y_train.append(torch.from_numpy(player['fantasy_scores'][:10]).view(1, 10, 1).float())
    Y_test.append(torch.from_numpy(player['fantasy_scores'][-5:]).view(1, 5, 1).float())

# Combine inputs and targets for all players
X_train = torch.cat(X_train, dim=0)
X_test = torch.cat(X_test, dim=0)
Y_train = torch.cat(Y_train, dim=0)
Y_test = torch.cat(Y_test, dim=0)

In [99]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Set hyperparameters
input_size = 2
hidden_size = 50
learning_rate = 0.001
num_epochs = 100

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [104]:
# Training loop for each player
for player_inputs, player_targets in zip(X_train, Y_train):
    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(player_inputs)
        loss = criterion(outputs, player_targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for every epoch
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Make predictions for the player's test inputs
with torch.no_grad():
    player_predicted = model(X_test)

print() # blank line
# Print the predictions and original scores for the player
for i in range(len(X_test[0])):
    print(f"Defense Rank: {X_test[0][i][0]}, Predicted Fantasy Score: {player_predicted[0][i].item()}, Actual Fantasy Score: {Y_test[0][i].item()}")
    print(f"Defense Rank: {X_test[1][i][0]}, Predicted Fantasy Score: {player_predicted[1][i].item()}, Actual Fantasy Score: {Y_test[1][i].item()}\n")


Epoch 1/100, Loss: 40.46646499633789
Epoch 2/100, Loss: 36.4416389465332
Epoch 3/100, Loss: 36.25300979614258
Epoch 4/100, Loss: 34.014373779296875
Epoch 5/100, Loss: 37.03624725341797
Epoch 6/100, Loss: 33.544925689697266
Epoch 7/100, Loss: 35.59321212768555
Epoch 8/100, Loss: 28.135990142822266
Epoch 9/100, Loss: 28.676305770874023
Epoch 10/100, Loss: 24.758602142333984
Epoch 11/100, Loss: 24.782360076904297
Epoch 12/100, Loss: 23.20244789123535
Epoch 13/100, Loss: 23.05584716796875
Epoch 14/100, Loss: 22.387479782104492
Epoch 15/100, Loss: 22.224977493286133
Epoch 16/100, Loss: 21.948436737060547
Epoch 17/100, Loss: 21.834461212158203
Epoch 18/100, Loss: 21.71729850769043
Epoch 19/100, Loss: 21.654857635498047
Epoch 20/100, Loss: 21.598796844482422
Epoch 21/100, Loss: 21.562313079833984
Epoch 22/100, Loss: 21.530914306640625
Epoch 23/100, Loss: 21.50701332092285
Epoch 24/100, Loss: 21.486095428466797
Epoch 25/100, Loss: 21.468326568603516
Epoch 26/100, Loss: 21.452213287353516
Epoch